# **Penting**
- Pastikan Anda melakukan Run All sebelum mengirimkan submission untuk memastikan seluruh cell berjalan dengan baik.
- Hapus simbol pagar (#) jika Anda menerapkan kriteria tambahan
- Biarkan simbol pagar (#) jika Anda tidak menerapkan kriteria tambahan

# **1. Import Library**
Pada tahap ini, Anda perlu mengimpor beberapa pustaka (library) Python yang dibutuhkan untuk analisis data dan pembangunan model machine learning.

In [30]:
#Type your code here
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier

# **2. Memuat Dataset dari Hasil Clustering**
Memuat dataset hasil clustering dari file CSV ke dalam variabel DataFrame.

In [9]:
# Gunakan dataset hasil clustering yang memiliki fitur Target
# Silakan gunakan dataset data_clustering jika tidak menerapkan Interpretasi Hasil Clustering [Advanced]
# Silakan gunakan dataset data_clustering_inverse jika menerapkan Interpretasi Hasil Clustering [Advanced]
df = pd.read_csv("data_clustering_inverse.csv")

In [10]:
# Tampilkan 5 baris pertama dengan function head.
df.head()

,TransactionType,Location,Channel,CustomerOccupation,muda,paruh baya,tua,TransactionAmount,CustomerAge,TransactionDuration,LoginAttempts,AccountBalance,Target
0,Debit,San Diego,ATM,Doctor,0,0,1,14.09,70.0,81.0,1.0,5112.21,0
1,Debit,Houston,ATM,Doctor,0,0,1,376.24,68.0,141.0,1.0,13758.91,2
2,Debit,Mesa,Online,Student,1,0,0,126.29,19.0,56.0,1.0,1122.35,2
3,Debit,Raleigh,Online,Student,1,0,0,184.50,26.0,25.0,1.0,8569.06,0
4,Debit,Oklahoma City,ATM,Student,1,0,0,92.15,18.0,172.0,1.0,781.68,0


In [16]:
df_kategorikal = df[["TransactionType", "Location",	"Channel", "CustomerOccupation"]].copy()
labels = []
encoders = {}

for label, content in df_kategorikal.items():
  labels.append(label)
  encoder = LabelEncoder()
  df_kategorikal[label] = encoder.fit_transform(content)
  encoders[label] = encoder

df = pd.concat([df_kategorikal, df[df.columns[4:]]], axis=1)
df

,TransactionType,Location,Channel,CustomerOccupation,muda,paruh baya,tua,TransactionAmount,CustomerAge,TransactionDuration,LoginAttempts,AccountBalance,Target
0,0,36,0,0,0,0,1,14.09,70.0,81.0,1.0,5112.21,0
1,0,15,0,0,0,0,1,376.24,68.0,141.0,1.0,13758.91,2
2,0,23,2,3,1,0,0,126.29,19.0,56.0,1.0,1122.35,2
3,0,33,2,3,1,0,0,184.50,26.0,25.0,1.0,8569.06,0
4,0,28,0,3,1,0,0,92.15,18.0,172.0,1.0,781.68,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1657,0,40,1,0,0,1,0,251.54,48.0,177.0,1.0,254.75,0
1658,0,36,1,2,0,0,1,28.63,56.0,146.0,1.0,3382.91,0
1659,0,10,2,3,1,0,0,185.97,23.0,19.0,1.0,1776.91,1
1660,0,27,1,2,0,0,1,12.18,76.0,77.0,1.0,4909.24,2


# **3. Data Splitting**
Tahap Data Splitting bertujuan untuk memisahkan dataset menjadi dua bagian: data latih (training set) dan data uji (test set).

In [17]:
# Menggunakan train_test_split() untuk melakukan pembagian dataset.
X = df.drop("Target", axis=1)
y = df["Target"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((1329, 12), (333, 12), (1329,), (333,))

# **4. Membangun Model Klasifikasi**
Setelah memilih algoritma klasifikasi yang sesuai, langkah selanjutnya adalah melatih model menggunakan data latih.

Berikut adalah rekomendasi tahapannya.
1. Menggunakan algoritma klasifikasi yaitu Decision Tree.
2. Latih model menggunakan data yang sudah dipisah.

In [18]:
# Buatlah model klasifikasi menggunakan Decision Tree
model = DecisionTreeClassifier()
model.fit(X_train, y_train)

DecisionTreeClassifier()

In [29]:
y_preds = model.predict(X_test)
print(classification_report(y_test, y_preds))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       113
           1       1.00      1.00      1.00       114
           2       1.00      1.00      1.00       106

    accuracy                           1.00       333
   macro avg       1.00      1.00      1.00       333
weighted avg       1.00      1.00      1.00       333



In [19]:
# Menyimpan Model
import joblib
joblib.dump(model, 'decision_tree_model.h5')

['decision_tree_model.h5']

# **5. Memenuhi Kriteria Skilled dan Advanced dalam Membangun Model Klasifikasi**



**Biarkan kosong jika tidak menerapkan kriteria skilled atau advanced**

In [23]:
# Melatih model menggunakan algoritma klasifikasi selain Decision Tree.
model_2 = RandomForestClassifier()
model_2.fit(X_train, y_train)

RandomForestClassifier()

In [27]:
# Menampilkan hasil evaluasi akurasi, presisi, recall, dan F1-Score pada seluruh algoritma yang sudah dibuat.
y_preds = model_2.predict(X_test)
print(classification_report(y_test, y_preds))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       113
           1       1.00      1.00      1.00       114
           2       1.00      1.00      1.00       106

    accuracy                           1.00       333
   macro avg       1.00      1.00      1.00       333
weighted avg       1.00      1.00      1.00       333



In [28]:
# Menyimpan Model Selain Decision Tree
# Model ini bisa lebih dari satu
import joblib
joblib.dump(model_2, 'explore_RandomForestClassifier_classification.h5')

['explore_RandomForestClassifier_classification.h5']

Hyperparameter Tuning Model

Pilih salah satu algoritma yang ingin Anda tuning

In [32]:
# Lakukan Hyperparameter Tuning dan Latih ulang.
random_grid = {
    'n_estimators' : [50, 75, 100],
    'max_depth' : [5, 10, 15],
    'min_samples_split' : [2, 5, 7],
    'min_samples_leaf' : [1, 2, 3],
    'max_features' : ['sqrt', 'log2'],
    'bootstrap' : [True, False]
}

rs_cv = RandomizedSearchCV(estimator=model_2,
                           param_distributions=random_grid,
                           n_iter=50,
                           cv=5,
                           verbose=1,
                           random_state=42,
                           n_jobs=-1)

rs_cv.fit(X_train, y_train)

Fitting 5 folds for each of 50 candidates, totalling 250 fits


RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(), n_iter=50,
                   n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [5, 10, 15],
                                        'max_features': ['sqrt', 'log2'],
                                        'min_samples_leaf': [1, 2, 3],
                                        'min_samples_split': [2, 5, 7],
                                        'n_estimators': [50, 75, 100]},
                   random_state=42, verbose=1)

In [33]:
print("parameter terbaik",rs_cv.best_params_)

parameter terbaik {'n_estimators': 50, 'min_samples_split': 7, 'min_samples_leaf': 3, 'max_features': 'sqrt', 'max_depth': 15, 'bootstrap': True}


In [34]:
model_3 = RandomForestClassifier(n_estimators=50,
                                 min_samples_split=7,
                                 min_samples_leaf=3,
                                 max_features='sqrt',
                                 max_depth=15,
                                 bootstrap=True)
model_3.fit(X_train, y_train)

RandomForestClassifier(max_depth=15, min_samples_leaf=3, min_samples_split=7,
                       n_estimators=50)

In [35]:
# Menampilkan hasil evaluasi akurasi, presisi, recall, dan F1-Score pada algoritma yang sudah dituning.
y_preds = model_3.predict(X_test)
print(classification_report(y_test, y_preds))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       113
           1       1.00      1.00      1.00       114
           2       1.00      1.00      1.00       106

    accuracy                           1.00       333
   macro avg       1.00      1.00      1.00       333
weighted avg       1.00      1.00      1.00       333



In [36]:
# Menyimpan Model hasil tuning
import joblib
joblib.dump(model_3, 'tuning_classification.h5')

['tuning_classification.h5']